In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from selenium.webdriver.chrome.options import Options

In [2]:
def fill_form(driver, name, gender, day, month, year, hour, minute, ampm, location):

    # Fill the 'Name' field
    name_field = driver.find_element(By.ID, "fin_name")
    name_field.clear()  # Clear any pre-filled text
    name_field.send_keys(name)

    # Fill the 'Gender' field 
    if gender.lower() == "male":
        gender_radio = driver.find_element(By.XPATH, '//input[@name="gender" and @value="male"]')
    elif gender.lower() == "female":
        gender_radio = driver.find_element(By.XPATH, '//input[@name="gender" and @value="female"]')
    gender_radio.click()

    # Fill the 'Day' field
    day_select = Select(driver.find_element(By.ID, "fin_day"))
    day_select.select_by_value(str(day))

    # Fill the 'Month' field
    month_select = Select(driver.find_element(By.ID, "fin_month"))
    month_select.select_by_value(str(month))

    # Fill the 'Year' field
    year_select = Select(driver.find_element(By.ID, "fin_year"))
    year_select.select_by_value(str(year))

    # Fill the 'Hour' field
    hour_select = Select(driver.find_element(By.ID, "fin_hour"))
    hour_select.select_by_value(str(hour))

    # Fill the 'Minute' field
    minute_select = Select(driver.find_element(By.ID, "fin_min"))
    minute_select.select_by_value(str(minute))

    # Fill the 'AM/PM' field
    ampm_select = Select(driver.find_element(By.ID, "fin_apm"))
    ampm_select.select_by_value(ampm)

    # Fill the 'Location' field
    location_input = driver.find_element(By.ID, "fin_location")
    
    location_input.clear()
    location_input.send_keys(location)
    time.sleep(2)  

    location_input.send_keys(Keys.DOWN) 
    time.sleep(1)
    location_input.send_keys(Keys.RETURN)  

    # Submit the form
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and contains(@class, 'btn-horoscope-form-submit')]")
    submit_button.click()

    # Wait for the page to load after submission 
    time.sleep(5)

In [3]:
def scrape_planet_positions(driver):

    table_rows = driver.find_elements(By.XPATH, "//table[@class='table table-bordered t-sm no-margin']/tbody/tr")
    
    planet_positions = []
    
    for row in table_rows:
        columns = row.find_elements(By.TAG_NAME, "td")
        
        if len(columns) == 6:
            planet = row.find_element(By.XPATH, ".//th").text.strip().split()[0]  
            position = columns[0].text.strip()
            degrees = columns[1].text.strip()
            rasi = columns[2].text.strip().split()[0]  
            rasi_lord = columns[3].text.strip()
            nakshatra = columns[4].text.strip()
            nakshatra_lord = columns[5].text.strip()
            
            planet_positions.append({
                "Planet": planet,
                "Position": position,
                "Degrees": degrees,
                "Rasi": rasi,
                "Rasi Lord": rasi_lord,
                "Nakshatra": nakshatra,
                "Nakshatra Lord": nakshatra_lord
            })
    
    planet_positions_df = pd.DataFrame(planet_positions)
    return planet_positions_df

In [5]:
if __name__ == "__main__":
    chrome_options = Options()
    chrome_options.add_argument("--headless")         # This makes the browser run in the background
    # chrome_options.add_argument("--disable-gpu")    # Disable GPU acceleration (optional)
    # chrome_options.add_argument("--no-sandbox")     # Fixes issues on some systems (optional)

    driver = webdriver.Chrome(options=chrome_options)

    # Open the webpage
    driver.get("https://www.prokerala.com/astrology/birth-chart/")

    # Wait for the page to load
    time.sleep(2)

    # Enter details 
    name = input("Enter your name: ")
    gender = input("Enter your gender (male/female): ")
    day = input("Enter your birth day (1-31): ")
    month = input("Enter your birth month (1-12): ")
    year = input("Enter your birth year (e.g., 1990): ")
    hour = input("Enter your birth hour (1-12): ")
    minute = input("Enter your birth minute (0-59): ")
    ampm = input("Enter AM/PM (am/pm): ")
    location = input("Enter your birth place (city, state, country): ")

    fill_form(driver, name, gender, day, month, year, hour, minute, ampm, location)


    planet_positions_df = scrape_planet_positions(driver)
    print(planet_positions_df)
    planet_positions_df.to_csv("planet_positions.csv", index=False)

    driver.quit()

      Planet  Position  Degrees       Rasi Rasi Lord         Nakshatra  \
0        Sun  256° 18'  16° 18′      Dhanu   Jupiter     Purva Ashadha   
1       Moon   94° 31'   4° 31′      Karka      Moon            Pushya   
2    Mercury  271° 24'   1° 24′     Makara    Saturn    Uttara Ashadha   
3      Venus   253° 2'   13° 2′      Dhanu   Jupiter             Moola   
4       Mars  322° 51'  22° 51′     Kumbha    Saturn  Purva Bhadrapada   
5    Jupiter   76° 48'  16° 48′    Mithuna   Mercury             Ardra   
6     Saturn   45° 27'  15° 27′  Vrishabha     Venus            Rohini   
7  Ascendant   179° 3'   29° 3′      Kanya   Mercury            Chitra   
8       Rahu   62° 28'   2° 28′    Mithuna   Mercury      Mrigashirsha   
9       Ketu  242° 28'   2° 28′      Dhanu   Jupiter             Moola   

  Nakshatra Lord  
0          Venus  
1         Saturn  
2            Sun  
3           Ketu  
4        Jupiter  
5           Rahu  
6           Moon  
7           Mars  
8           Ma